In [ ]:
# pour les imports
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
n=7
m=11
points_1_10=[2, 3, 4, 5, 6, 7, 8, 9, 10] # without 1 (the enemy should choose from 2 to 10)
maxPointsEnemmy=100 # the maximum number of points of each palyer
maxPointsAgent=100

# Zayed

In [ ]:
class Cards:
    def __init__(self):
        self.ncards = (m-n+1)*4  # number of cards
        self.cards = 4*[i for i in range(n,m+1)]  # cards 

    def shuffle(self):
        '''Knuth shuffle'''
        np.random.shuffle(self.cards)

    def hands(self):
        '''distribution of cards by the agent :)'''
        self.hand1 = [self.cards[0], self.cards[2]]
        self.hand2 = [self.cards[1], self.cards[3]]



In [ ]:
class Enemies :
    '''many enemies'''

    def __init__(self):
        self.maxPoints= maxPointsEnemmy # the maximum amount of points that the enemy has
        self.points= points_1_10
        # self.dictionnary = {}
        
        # for i in range(2,11):
        #     for j in range(14, 23):
        #         self.dictionnary[str(j)+"-"+str(i)] = {}
        #         self.dictionnary[str(j)+"-"+str(i)][False] = 0
        #         self.dictionnary[str(j)+"-"+str(i)][True] = 0
        
    # def abdel(self, hand, gofor):
    #     if np.random.random() < .05:
    #         action =  np.random.random()<.5
    #         if self.dictionnary.get(str(sum(hand))+"-"+str(gofor)) is None:
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)] = {}
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)][False] = 0
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)][True] = 0
    #         return action
        
    #     try:
    #         return max(self.dictionnary.get(str(sum(hand))+"-"+str(gofor)))
        
    #     except:
    #         action = np.random.random()<.5

    #         if self.dictionnary.get(str(sum(hand))+"-"+str(gofor)) is None:
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)] = {}
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)][False] = 0
    #             self.dictionnary[str(sum(hand))+"-"+str(gofor)][True] = 0
                
    #         return action
            
        
        

    
    def enemy1(self, hand1): # the enemy hand
        goforpoints = None
        if hand1[0]+hand1[1] >= 18:  # (the mean of all possible hands)
            goforpoints = np.random.choice(self.points[5:])
        else:
            goforpoints = np.random.choice(self.points[:5])
        return goforpoints
    def deterministic(self, hand1):
        sum=hand1[0]+hand1[1]
        goforpoints=None
        if(sum==14):
            goforpoints=2
        if(sum==15):
            goforpoints=3
        if(sum==16):
             goforpoints=4
        if(sum==17):
             goforpoints=5
        if(sum==18):
             goforpoints=6
        if(sum==19):
              goforpoints=7
        if(sum==20):
              goforpoints=8
        if(sum==21):
               goforpoints=9
        if(sum==22):
            goforpoints=10
        return goforpoints
    def Human(self, hand1):
        goforpoints = None

        while not (goforpoints in self.points):
            print("your hand ", hand1)
            goforpoints = int(input("Input your action (int): ")) 

        return goforpoints
    


In [ ]:
def mapTo1422(sampled,a14,b22):

    return int((b22-a14)*sampled+a14) # the distribution of an arm (a point) ??

In [ ]:
class ThompsonSampling :
    def __init__(self,alist,blist):
        self.maxPoints= maxPointsAgent  # the maximum amount of points that the agent has
        # self.points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # we can keep track of this beta =>  the agent learn  ?????
        self.alist = alist
        self.blist = blist

    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        estimated =np.random.beta(self.alist[enemyResponse-1],self.blist[enemyResponse-1])

        estimatedSumEnemyHand=mapTo1422(estimated,14,22) # according to the agent technique
        if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
            return True # go for
        else :
            return False # go for -1
    def updateBeta(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        if(gowith) and h2<h1 :
            self.alist[enemyResponse-1]+=1 # shift bata to the right (we need to adapte the learning parameter)
        if  h1<=h2 and not (gowith):    
            self.blist[enemyResponse-1]+=1 # shift bata to the left




    

In [ ]:
# test
enemy=Enemies()
#b=[1 for i in range(10)]
#a=[1 for i in range(10)]
agent=ThompsonSampling(a,b)
while(enemy.maxPoints>=10 and agent.maxPoints>=10):
    cards = Cards()
    cards.shuffle()
    cards.hands()
    hand1= cards.hand1
    hand2=cards.hand2
    EnemyGoFor=enemy.Human(hand1)
    AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
    #show cards
    if(AgentGo):
        if(np.sum(hand1)>np.sum(hand2)):
            # agent loses
            agent.maxPoints-=EnemyGoFor
        if(np.sum(hand2)>np.sum(hand1)):
            # agent loses
            enemy.maxPoints-=EnemyGoFor
    else :
        agent.maxPoints-=2
    # update agent strategy
    agent.updateBeta(AgentGo,EnemyGoFor,hand1,hand2)
    print("=========Enemy========")
    print("hand1 ", hand1)
    print("GoFor", EnemyGoFor)
    print("maxPoints= ", enemy.maxPoints)
    print("=========Agent========")
    print("hand2 ", hand2)
    print("GoFor",AgentGo)
    print("maxPoints= ", agent.maxPoints)
if(enemy.maxPoints <10):
    print("Enemy losed")
if(agent.maxPoints <10):
    print("Agent losed")




In [ ]:
a=agent.alist
b=agent.blist 

In [ ]:
a=agent.alist
b=agent.blist
# print(agent.alist)
# print(agent.blist)
x=np.linspace(0,1,50)
x_=np.linspace(14,22,50)

for i in range(10):
    y=y = ss.beta.pdf (x, a[i], b[i])
    plt.plot(x_,(22-14)*y+14,label=f"{i+1}")
    plt.legend()
    plt.show()

In [ ]:
from tqdm import trange

In [ ]:
# test

# enemy=Enemies()
# enemy1 = Enemies()
# lr = .2
# l = []
# for k in trange(100):
#     score = 0
    
#     for i in range(100):
#         b=[1 for i in range(10)]
#         a=[1 for i in range(10)]
#         agent=ThompsonSampling(a,b)
#         while(enemy.maxPoints>=10 and agent.maxPoints>=10):
#             cards = Cards()
#             cards.shuffle()
#             cards.hands()
#             hand1 = cards.hand1
#             hand2 = cards.hand2
#             EnemyGoFor=enemy1.deterministic(hand1)
#             AgentGo=enemy.abdel(hand2,EnemyGoFor)
#             #show cards
#             if(AgentGo):
#                 if(np.sum(hand1)>np.sum(hand2)):
#                     # agent loses
#                     enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][AgentGo] += lr*(EnemyGoFor - enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][AgentGo]) 
#                     agent.maxPoints-=EnemyGoFor
#                 if(np.sum(hand2)>np.sum(hand1)):
#                     # agent loses
#                     enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][AgentGo] += lr*(-EnemyGoFor - enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][AgentGo]) 
#                     enemy.maxPoints-=EnemyGoFor
#             else :
#                 enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][False] += lr*( - 1 - enemy.dictionnary[str(sum(hand1))+"-"+str(EnemyGoFor)][False]) 
#                 enemy.maxPoints-=1
#             # update agent strategy
        
#         if(enemy.maxPoints <10):
#             score += 1.
#         enemy.maxPoints = 100
#     l.append(score)

In [ ]:
# enemy.dictionnary

In [ ]:
# plt.plot(l)

In [ ]:
# b=[1 for i in range(10)]
# a=[1 for i in range(10)]
# enemy.maxPoints = 100
# agent=ThompsonSampling(a,b)
# while(enemy.maxPoints>=10 and agent.maxPoints>=10):
#     cards = Cards()
#     cards.shuffle()
#     cards.hands()
#     hand1 = cards.hand1
#     hand2 = cards.hand2
#     EnemyGoFor=int(enemy.abdel(hand1))
#     AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
#     #show cards
#     if(AgentGo):
#         if(np.sum(hand1)>np.sum(hand2)):
#             # agent loses
#             enemy.dictionnary[str(sum(hand1))][str(EnemyGoFor)] += lr*(EnemyGoFor - enemy.dictionnary[str(sum(hand1))][str(EnemyGoFor)]) 
#             agent.maxPoints-=EnemyGoFor
#         if(np.sum(hand2)>np.sum(hand1)):
#             # agent loses
#             enemy.dictionnary[str(sum(hand1))][str(EnemyGoFor)] += lr*(-EnemyGoFor - enemy.dictionnary[str(sum(hand1))][str(EnemyGoFor)]) 
#             enemy.maxPoints-=EnemyGoFor
#     else :
#         agent.maxPoints-=1
        
#     agent.updateBeta(AgentGo,EnemyGoFor,hand1,hand2)

In [ ]:
# a=agent.alist
# b=agent.blist
# # print(agent.alist)
# # print(agent.blist)
# x=np.linspace(0,1,50)
# x_=np.linspace(14,22,50)

# for i in range(10):
#     y=y = ss.beta.pdf (x, a[i], b[i])
#     plt.plot(x_,(22-14)*y+14,label=f"{i+1}")
#     plt.legend()
#     plt.show()